In [1]:
import sys
import os

import matplotlib.pyplot as plt
#%matplotlib inline
import numpy as np

from few.trajectory.inspiral import EMRIInspiral
from few.amplitude.romannet import RomanAmplitude
from few.amplitude.interp2dcubicspline import Interp2DAmplitude
from few.waveform import FastSchwarzschildEccentricFlux, SlowSchwarzschildEccentricFlux, GenerateEMRIWaveform
from few.utils.utility import (get_overlap, 
                               get_mismatch, 
                               get_fundamental_frequencies, 
                               get_separatrix, 
                               get_mu_at_t, 
                               get_p_at_t, 
                               get_kerr_geo_constants_of_motion,
                               xI_to_Y,
                               Y_to_xI)

from few.utils.ylm import GetYlms
from few.utils.modeselector import ModeSelector
from few.summation.interpolatedmodesum import CubicSplineInterpolant
from few.waveform import SchwarzschildEccentricWaveformBase
from few.summation.interpolatedmodesum import InterpolatedModeSum
from few.summation.directmodesum import DirectModeSum
from few.utils.constants import *
from few.summation.aakwave import AAKSummation
from few.waveform import Pn5AAKWaveform, AAKWaveformBase

from MOO_func import tukey
from MOO_func import inner_prod
from MOO_func import PSD_Lisa
from MOO_func import PSD_Taiji
from MOO_func import PSD_Tianqin
from MOO_func import PSD_armlength_dependent
from MOO_func import Find_zero
from MOO_func import E
from MOO_func import DL
from MOO_func import PSD_L_lambda
from MOO_func import SNR_for_diff_para
from MOO_func import SNR_M_D_L_l
from MOO_func import htilde
from MOO_func import T_chirp
from MOO_func import final_frequency
from MOO_func import SNR_binary_redshift
from scipy.optimize import brentq
from MOO_func import secant_method
use_gpu = False

# keyword arguments for inspiral generator (RunSchwarzEccFluxInspiral)
inspiral_kwargs={
        "DENSE_STEPPING": 0,  # we want a sparsely sampled trajectory
        "max_init_len": int(1e3),  # all of the trajectories will be well under len = 1000
    }

# keyword arguments for inspiral generator (RomanAmplitude)
amplitude_kwargs = {
    "max_init_len": int(1e3),  # all of the trajectories will be well under len = 1000
    "use_gpu": use_gpu  # GPU is available in this class
}

# keyword arguments for Ylm generator (GetYlms)
Ylm_kwargs = {
    "assume_positive_m": False  # if we assume positive m, it will generate negative m for all m>0
}

# keyword arguments for summation generator (InterpolatedModeSum)
sum_kwargs = {
    "use_gpu": use_gpu,  # GPU is availabel for this type of summation
    "pad_output": False,
}

# set omp threads one of two ways
num_threads = 4

# this is the general way to set it for all computations
# from few.utils.utility import omp_set_num_threads
# omp_set_num_threads(num_threads)

few = FastSchwarzschildEccentricFlux(
    inspiral_kwargs=inspiral_kwargs,
    amplitude_kwargs=amplitude_kwargs,
    Ylm_kwargs=Ylm_kwargs,
    sum_kwargs=sum_kwargs,
    use_gpu=use_gpu,
    num_threads=num_threads,  # 2nd way for specific classes
)

gen_wave = GenerateEMRIWaveform("Pn5AAKWaveform")

# parameters
T = 1 # years
dt = 10  # seconds
M = 5e5
a = 0.98
mu = 50
p0 = 11.0
e0 = 0.1
x0 = 0.7  # notice this is x_I, not Y. The AAK waveform can convert to Y. 
qK = 0.2  # polar spin angle
phiK = 0.2  # azimuthal viewing angle
qS = 0.3  # polar sky angle
phiS = 0.3  # azimuthal viewing angle
dist = 10# distance
Phi_phi0 = 1.0
Phi_theta0 = 2.0
Phi_r0 = 3.0

# h = gen_wave(
#     M,
#     mu,
#     a,
#     p0,
#     e0,
#     x0,
#     dist,
#     qS,
#     phiS,
#     qK,
#     phiK,
#     Phi_phi0,
#     Phi_theta0,
#     Phi_r0,
#     T=T,
#     dt=dt,
# )

# plt.plot(h.real[:2000])
# plt.show

# # temp=h[:2000]
# # h=temp
# print(len(h))

# para=np.array([M,mu,a,p0,e0,x0,dist,qS,phiS,qK,phiK,Phi_phi0,Phi_theta0,Phi_r0,T,dt])
# print(para)
# temp=str(para)
# with open('parametersAAK_PN5.txt', 'w') as f:
#     f.write(temp)

import numpy as np
import matplotlib.pyplot as plt
from scipy.fftpack import fft, ifft,fftfreq
import math
from math import pi as Pi



# para=np.array([M,mu,a,p0,e0,x0,dist,qS,phiS,qK,phiK,Phi_phi0,Phi_theta0,Phi_r0,T,dt])
# temp=para
# #para=[float(x) for x in temp]
# # temp=h[-10000:]
# # h=temp
# wave1 = np.array(h)
# f = np.array(np.arange(len(h))/dt/ len(h))
# plt.loglog(f,wave1.real,'ro')
# plt.show()

# tukey_seq=[tukey(i,len(h),1/8) for i in range(0,len(h))]
# wave1 = tukey_seq*wave1

# waveform1 = fft(wave1)
# waveform2 = np.column_stack((waveform1, f))
# temp=waveform2.real*waveform2.real+waveform2.imag*waveform2.imag
# waveform = np.sqrt(temp)


# plt.loglog(f, np.abs(waveform1), 'ro')
# plt.show()

# np.savetxt("waveform.txt", waveform, fmt="%50.50f", delimiter=" ")

# import re
# with open('waveform.txt', 'r') as f:
#     text = f.read()
#     patn = re.sub(r"[\([{})\]]", "", text)

# with open('waveformAAK_PN5.txt', 'w') as f:
#     f.write(patn)

GM_sun = 1.3271244*1e20 #   这个式子等于  G * M_sun
c =2.9979246*1e8
M_sun =1.9884099*1e30
G = 6.6743*1e-11
pc= 3.0856776*1e16

Mpc = (10**6) * pc


#选择一个质量计算探测距离随臂长关系
M_final=10**5
L1=5*10**8
L2=10*10**9
dL=1*10**8
l1=500
l2=1200
dl=100
t0 =1.
phi0 =0.
fmin = 1e-4

# variables to sample through

m1 = 1e5  
m2 = 2*1e5

state1=[0]
np.savetxt("state1.txt",state1,fmt="%50.50f",delimiter=" ")
itr=100000
e=1
# z1=0.000001
# z2=100
z1=0.1
z2=20
dz=0.01
log_z1=np.log(z1)
log_z2=np.log(z2)
dlog_z=0.5

L1=5*10**8
L2=10*10**9
dL=5*10**8
l1=200
l2=1200
dl=100
# SNRlist2=SNR_for_diff_para(freq_bin,h_f,PSD_L_lambda,L1,L2,dL,l1,l2,dl,figure_file=None)
from scipy import interpolate
# np.savetxt("SNR_binary.txt",SNRlist2,fmt="%50.50f",delimiter=" ")
z_line=np.exp(np.arange(log_z1,log_z2,dlog_z))
#SNR_line=np.zeros(len(z_line))
zseq=np.zeros((len(np.arange(L1,L2,dL)),len(np.arange(l1,l2,dl))))

SNR_threshold=50
zseq=np.zeros((len(np.arange(L1,L2,dL)),len(np.arange(l1,l2,dl))))
i=0
j=0


# L1=2*10**9
# L2=4*10**9
# dL=1*10**8
# l1=200
# l2=1800
# dl=50
# SNR_threshold=10
# zseq=np.zeros((len(np.arange(L1,L2,dL)),len(np.arange(l1,l2,dl))))
# i=0
# j=0
# for L in np.arange(L1,L2,dL):
#     for l in np.arange(l1,l2,dl):
#         f=lambda x:SNR_binary_redshift(m1,m2,L,l,fmin,x)-SNR_threshold
#         zeropoint=brentq(f,z1,z2)
#         if zeropoint>0:
#             zseq[i][j]=zeropoint
#         j=j+1
#     j=0
#     i=i+1
# np.savetxt("redshift_binary2.txt",zseq,fmt="%50.50f",delimiter=" ")
# state1=[2]
# np.savetxt("state1.txt",state1,fmt="%50.50f",delimiter=" ")


# L1=4*10**9
# L2=6*10**9
# dL=1*10**8
# l1=200
# l2=1800
# dl=50
# SNR_threshold=10
# zseq=np.zeros((len(np.arange(L1,L2,dL)),len(np.arange(l1,l2,dl))))
# i=0
# j=0
# for L in np.arange(L1,L2,dL):
#     for l in np.arange(l1,l2,dl):
#         f=lambda x:SNR_binary_redshift(m1,m2,L,l,fmin,x)-SNR_threshold
#         zeropoint=brentq(f,z1,z2)
#         if zeropoint>0:
#             zseq[i][j]=zeropoint
#         j=j+1
#     j=0
#     i=i+1
# np.savetxt("redshift_binary3.txt",zseq,fmt="%50.50f",delimiter=" ")
# state1=[3]
# np.savetxt("state1.txt",state1,fmt="%50.50f",delimiter=" ")


# L1=6*10**9
# L2=8*10**9
# dL=1*10**8
# l1=200
# l2=1800
# dl=50
# SNR_threshold=10
# zseq=np.zeros((len(np.arange(L1,L2,dL)),len(np.arange(l1,l2,dl))))
# i=0
# j=0
# for L in np.arange(L1,L2,dL):
#     for l in np.arange(l1,l2,dl):
#         f=lambda x:SNR_binary_redshift(m1,m2,L,l,fmin,x)-SNR_threshold
#         zeropoint=brentq(f,z1,z2)
#         if zeropoint>0:
#             zseq[i][j]=zeropoint
#         j=j+1
#     j=0
#     i=i+1
# np.savetxt("redshift_binary4.txt",zseq,fmt="%50.50f",delimiter=" ")
# state1=[4]
# np.savetxt("state1.txt",state1,fmt="%50.50f",delimiter=" ")


# L1=8*10**9
# L2=10*10**9
# dL=1*10**8
# l1=200
# l2=1800
# dl=50
# SNR_threshold=10
# zseq=np.zeros((len(np.arange(L1,L2,dL)),len(np.arange(l1,l2,dl))))
# i=0
# j=0
# for L in np.arange(L1,L2,dL):
#     for l in np.arange(l1,l2,dl):
#         f=lambda x:SNR_binary_redshift(m1,m2,L,l,fmin,x)-SNR_threshold
#         zeropoint=brentq(f,z1,z2)
#         if zeropoint>0:
#             zseq[i][j]=zeropoint
#         j=j+1
#     j=0
#     i=i+1
# np.savetxt("redshift_binary5.txt",zseq,fmt="%50.50f",delimiter=" ")
# state1=[5]
# np.savetxt("state1.txt",state1,fmt="%50.50f",delimiter=" ")


# redshift1=np.loadtxt(r'redshift_binary1.txt')
# redshift2=np.loadtxt(r'redshift_binary2.txt')
# redshift3=np.loadtxt(r'redshift_binary3.txt')
# redshift4=np.loadtxt(r'redshift_binary4.txt')
# redshift5=np.loadtxt(r'redshift_binary5.txt')
# redshift=np.row_stack((redshift1,redshift2,redshift3,redshift4,redshift5))
# np.savetxt("redshift_binary.txt",redshift,fmt="%50.50f",delimiter=" ")

In [2]:
L1=5*10**8
L2=2*10**9
dL=5*10**8
l1=200
l2=1200
dl=100
for L in np.arange(L1,L2,dL):
    for l in np.arange(l1,l2,dl):
        print(L,l)
        print(SNR_binary_redshift(m1,m2,L,l,fmin,z1))
        print(SNR_binary_redshift(m1,m2,L,l,fmin,z2))
        SNR_line=np.zeros(len(z_line))
        k=0
        for k in range(len(z_line)):
            SNR_line[k]=SNR_binary_redshift(m1,m2,L,l,fmin,z_line[k])
        cubic_interp = interpolate.interp1d(z_line, SNR_line, kind='cubic')
        f=lambda x:SNR_binary_redshift(m1,m2,L,l,fmin,x)-SNR_threshold
        print(f(z1+0.1*z1),f(z2-0.1*z2))

        zeropoint=brentq(f,z1+0.1*z1,z2-0.1*z2)
        
        
        
        if zeropoint  > 0:
            zseq[i][j]=zeropoint
            print('done')
        j+=1
    np.savetxt("/home/ljq/code/Multi-Obj-Opt2.0/results/redshift/binary/redshift_binary_1.txt",zseq,fmt="%50.50f",delimiter=" ")
    j=0
    i+=1
np.savetxt("/home/ljq/code/Multi-Obj-Opt2.0/results/redshift/binary/redshift_binary_1.txt",zseq,fmt="%50.50f",delimiter=" ")
state1=[1]
np.savetxt("state1.txt",state1,fmt="%50.50f",delimiter=" ")


500000000 200
2433.1592931142222
5.051188373784072
2147.5260529922357 -44.32516591373234
done
500000000 300
3541.642341164604
7.352376257657873
3148.660826323567 -41.739861530690945
done
500000000 400
4487.293136128622
9.31552775156104
4002.732429792776 -39.53432925002409
done
500000000 500
5234.095175688978
10.865873341519254
4677.212289375299 -37.79257001470243
done
500000000 600
5789.767287379785
12.019437153864772
5179.071722013892 -36.49658127728797
done
500000000 700
6187.274073557583
12.844653021368076
5538.082955408932 -35.56948001182311
done
500000000 800
6463.918683033019
13.418961509516128
5787.9366080085165 -34.92426444205917
done
500000000 900
6651.115451900045
13.80757813037975
5957.004772918945 -34.487667213174944
done
500000000 1000
6772.41003574008
14.059383178010027
6066.552885461286 -34.204773018453324
done
500000000 1100
6844.747173362106
14.20955357975153
6131.884713497835 -34.03606180606273
done
1000000000 200
4806.448419260697
9.978087519082745
4290.979915938787 

In [4]:
L1=2.0*10**9
L2=4.0*10**9
dL=5.0*10**8
l1=200.0
l2=1200.0
dl=100.0

L=np.arange(L1,L2,dL)

l=np.arange(l1,l2,dl)

i = 0
while i < len(L):
    j = 0
    while j < len(l):
        print(L[i], l[j])
        print(m1,m2,fmin,z1,z2)
        print(SNR_binary_redshift(m1, m2, L[i], l[j], fmin, z1))
        print(SNR_binary_redshift(m1, m2, L[i], l[j], fmin, z2))
        
        SNR_line = np.zeros(len(z_line))
        k = 0
        while k < len(z_line):
            SNR_line[k] = SNR_binary_redshift(m1, m2, L[i], l[j], fmin, z_line[k])
            k += 1
        
        cubic_interp = interpolate.interp1d(z_line, SNR_line, kind='cubic')
        f = lambda x: SNR_binary_redshift(m1, m2, L[i], l[j], fmin, x) - SNR_threshold
        print(f(z1 + 0.1 * z1), f(z2 - 0.1 * z2))

        zeropoint = brentq(f, z1 + 0.1 * z1, z2 - 0.1 * z2)
        
        if zeropoint > 0:
            zseq[i][j] = zeropoint
            print('done')
        
        j += 1
    
    np.savetxt("/home/ljq/code/Multi-Obj-Opt2.0/results/redshift/binary/redshift_binary_2.txt", zseq, fmt="%50.50f", delimiter=" ")
    i += 1
# for i in np.arange(len(L)):
#     for j in np.arange(len(l)):
#         print(L[i],l[j])
#         print(SNR_binary_redshift(m1,m2,L[i],l[j],fmin,z1))
#         print(SNR_binary_redshift(m1,m2,L[i],l[j],fmin,z2))
#         SNR_line=np.zeros(len(z_line))
#         k=0
#         for k in range(len(z_line)):
#             SNR_line[k]=SNR_binary_redshift(m1,m2,L[i],l[j],fmin,z_line[k])
#         cubic_interp = interpolate.interp1d(z_line, SNR_line, kind='cubic')
#         f=lambda x:SNR_binary_redshift(m1,m2,L[i],l[j],fmin,x)-SNR_threshold
#         print(f(z1+0.1*z1),f(z2-0.1*z2))

#         zeropoint=brentq(f,z1+0.1*z1,z2-0.1*z2)
        
        
        
#         if zeropoint  > 0:
#             zseq[i][j]=zeropoint
#             print('done')
        
#     np.savetxt("/home/ljq/code/Multi-Obj-Opt2.0/results/redshift/binary/redshift_binary_2.txt",zseq,fmt="%50.50f",delimiter=" ")
    
np.savetxt("/home/ljq/code/Multi-Obj-Opt2.0/results/redshift/binary/redshift_binary_2.txt",zseq,fmt="%50.50f",delimiter=" ")

2000000000.0 200.0
100000.0 200000.0 0.0001 0.1 20
freq_bin的长度为 1482778
[0.0001     0.00010001 0.00010002 ... 0.01465723 0.01465724 0.01465725]
0.0 1.6724609985715867e-23 2.3289315774651355e-28
Sc: min=0.0, max=3.158997405723524e-35
Poms: min=1.6724609985715867e-23, max=2.6750269834898203e-18
Pacc: min=2.3289315774651355e-28, max=3.8250000933837895e-27
(4 * Pacc) / (np.power(2 * np.pi * f, 4)): 1.5360774130376466e-22
1 + 0.6 * (f / f0)**2: 1.000010542159114
freq_bin的长度为 1482778
[0.0001     0.00010001 0.00010002 ... 0.01465723 0.01465724 0.01465725]
0.0 1.6724609985715867e-23 2.3289315774651355e-28
Sc: min=0.0, max=3.158997405723524e-35
Poms: min=1.6724609985715867e-23, max=2.6750269834898203e-18
Pacc: min=2.3289315774651355e-28, max=3.8250000933837895e-27
(4 * Pacc) / (np.power(2 * np.pi * f, 4)): 1.5360774130376466e-22
1 + 0.6 * (f / f0)**2: 1.000010542159114
9189.573513748495
freq_bin的长度为 1482778
[0.0001     0.00010001 0.00010002 ... 0.01465723 0.01465724 0.01465725]
0.0 1.6724609985

ValueError: f(a) and f(b) must have different signs

In [3]:
L=2000000000
l=200
print(L,l)
print(SNR_binary_redshift(m1,m2,L,l,fmin,z1))
print(SNR_binary_redshift(m1,m2,L,l,fmin,z2))
SNR_line=np.zeros(len(z_line))
k=0
for k in range(len(z_line)):
    SNR_line[k]=SNR_binary_redshift(m1,m2,L,l,fmin,z_line[k])
cubic_interp = interpolate.interp1d(z_line, SNR_line, kind='cubic')
f=lambda x:SNR_binary_redshift(m1,m2,L,l,fmin,x)-SNR_threshold
print(f(z1+0.1*z1),f(z2-0.1*z2))

zeropoint=brentq(f,z1+0.1*z1,z2-0.1*z2)



if zeropoint  > 0:
    zseq[i][j]=zeropoint
    print('done')




2000000000 200
freq_bin的长度为 1482778
[0.0001     0.00010001 0.00010002 ... 0.01465723 0.01465724 0.01465725]
0.0 1.6724609985715867e-23 2.3289315774651355e-28
Sc: min=0.0, max=3.158997405723524e-35
Poms: min=1.6724609985715867e-23, max=2.6750269834898203e-18
Pacc: min=2.3289315774651355e-28, max=3.8250000933837895e-27
(4 * Pacc) / (np.power(2 * np.pi * f, 4)): 1.5360774130376466e-22
1 + 0.6 * (f / f0)**2: 1.000010542159114
freq_bin的长度为 1482778
[0.0001     0.00010001 0.00010002 ... 0.01465723 0.01465724 0.01465725]
0.0 1.6724609985715867e-23 2.3289315774651355e-28
Sc: min=0.0, max=3.158997405723524e-35
Poms: min=1.6724609985715867e-23, max=2.6750269834898203e-18
Pacc: min=2.3289315774651355e-28, max=3.8250000933837895e-27
(4 * Pacc) / (np.power(2 * np.pi * f, 4)): 1.5360774130376466e-22
1 + 0.6 * (f / f0)**2: 1.000010542159114
9189.573513748495
freq_bin的长度为 1482778
[0.0001     0.00010001 0.00010002 ... 0.01465723 0.01465724 0.01465725]
0.0 1.6724609985715867e-23 2.3289315774651355e-28
Sc

In [8]:
L=2000000000-dL
l=l2
print(L,l)
print(SNR_binary_redshift(m1,m2,L,l,fmin,z1))
print(SNR_binary_redshift(m1,m2,L,l,fmin,z2))
SNR_line=np.zeros(len(z_line))
k=0
for k in range(len(z_line)):
    SNR_line[k]=SNR_binary_redshift(m1,m2,L,l,fmin,z_line[k])
cubic_interp = interpolate.interp1d(z_line, SNR_line, kind='cubic')
f=lambda x:SNR_binary_redshift(m1,m2,L,l,fmin,x)-SNR_threshold
print(f(z1+0.1*z1),f(z2-0.1*z2))

zeropoint=brentq(f,z1+0.1*z1,z2-0.1*z2)



if zeropoint  > 0:
    zseq[i][j]=zeropoint
    print('done')


1500000000 1200
16378.596066328186
34.00162671768898
14742.451800854298 -11.800357458945285
done
